In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train_tmp, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [6]:
if len(instances_train_tmp) > config.size:
    instances_train = np.random.choice(instances_train_tmp, config.size, replace=False)
else:
    instances_train = instances_train_tmp

In [7]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [8]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [9]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [10]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [11]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    time_start = time.time()
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

    if epoch % 10 == 0:
        time_log = float(time.time() - time_start)
        
        # validate
        loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
        ppl_train = np.exp(np.mean(ppls_train))
        loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

        # test
        if ppl_dev < ppl_min:
            ppl_min = ppl_dev
            loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
            saver.save(sess, config.path_model, global_step=global_step_log)
            cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
            update_checkpoint(config, checkpoint, global_step_log)

        # visualize topic
        topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
        topics_freq_idxs = bow_idxs[topics_freq_indices]
        topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
        topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
        descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
        recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}

        depth_specs = get_topic_specialization(sess, model, instances_test)
        hierarchical_affinities = get_hierarchical_affinity(sess, model)

        # log
        clear_output()
        log_series = pd.Series([time_log, epoch, ct, \
                '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                '%.2f'%loss_dev, ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                '%.2f'%loss_test, ppl_test, \
                '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                index=log_df.columns)
        log_df.loc[global_step_log] = log_series
        display(log_df)
        cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
        print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

        # update tree
        if not config.static:
            config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
            if update_tree_flg:
                print(config.tree_idxs)
                name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                if 'sess' in globals(): sess.close()
                model = HierarchicalNeuralTopicModel(config)
                sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                saver = tf.train.Saver(max_to_keep=1)

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:              \
           Time   Ep  Ct    LOSS  PPL     NLL    KL   REG    LOSS         PPL   
31     2.768924    0  30  117.72  791  116.80  0.75  0.16  106.72  573.571716   
341    0.617986   10  30  112.66  593  112.25  0.39  0.02  105.61  537.981750   
651    0.300289   20  30  112.06  574  111.59  0.46  0.01  105.47  533.718018   
961    0.739563   30  30  111.63  561  111.03  0.59  0.01  105.29  530.653320   
1271   0.325155   40  30  111.30  552  110.57  0.72  0.01  105.20  530.028931   
1581   0.595456   50  30  111.03  545  110.20  0.83  0.01  105.17  529.619141   
1891   0.548240   60  30  110.81  540  109.89  0.92  0.01  105.22  531.926819   
2201   0.777141   70  30  110.63  535  109.63  1.00  0.01  105.21  531.568298   
2511   0.424271   80  30  110.48  531  109.40  1.07  0.01  105.29  534.278992   
2821   0.353124   90  30  110.34  527  109.20  1.14  0.01  105.31  533.932617   
3131   0.436757  100  30  110.22  524  109.02  1.20  0.01  105.38  536.555054   
3441   0.315981  110  30  110.11  521  108.85  1.25  0.01  105.43  537.874756   
3751   0.324044  120  30  110.01  519  108.70  1.30  0.01  105.47  539.184387   
4061   0.316536  130  30  109.92  517  108.56  1.35  0.01  105.51  540.470947   
4371   0.451363  140  30  109.83  514  108.44  1.39  0.01  105.55  541.683350   
4681   0.332863  150  30  109.76  512  108.32  1.43  0.01  105.58  542.823853   
4991   0.325603  160  30  109.68  511  108.21  1.47  0.01  105.61  543.923767   
5301   0.323954  170  30  109.61  509  108.11  1.50  0.01  105.64  544.965027   
5611   0.310067  180  30  109.55  507  108.01  1.53  0.01  105.67  545.849792   
5921   0.336730  190  30  109.49  506  107.92  1.56  0.01  105.69  546.707031   
6231   0.319263  200  30  109.43  504  107.83  1.59  0.01  105.72  547.496216   
6541   0.374295  210  30  109.38  503  107.75  1.62  0.01  105.74  548.197388   
6851   0.323386  220  30  109.32  502  107.67  1.65  0.01  105.76  548.825623   
7161   0.697526  230  30  109.27  500  107.60  1.67  0.00  105.78  549.404785   
7471   0.776065  240  30  109.22  499  107.53  1.69  0.00  105.81  549.948425   
7781   0.449024  250  30  109.18  498  107.46  1.71  0.00  105.83  550.464722   
8091   0.492245  260  30  109.13  497  107.39  1.73  0.00  105.85  550.964172   
8401   0.452463  270  30  109.09  496  107.33  1.75  0.00  105.88  551.453308   
8711   0.347524  280  30  109.05  495  107.27  1.77  0.00  105.90  551.928955   
9021   0.319563  290  30  109.01  494  107.21  1.79  0.00  105.92  552.394470   
...         ...  ...  ..     ...  ...     ...   ...   ...     ...         ...   
21731  0.342975  700  30  108.07  471  105.90  2.18  0.00  106.57  572.785645   
22041  0.323649  710  30  108.06  471  105.88  2.18  0.00  106.58  573.864685   
22351  0.320522  720  30  108.05  470  105.86  2.19  0.00  106.60  574.487854   
22661  0.362783  730  30  108.04  470  105.84  2.19  0.00  106.62  575.068298   
22971  0.460580  740  30  108.02  470  105.82  2.20  0.00  106.63  575.626343   
23281  0.388145  750  30  108.01  469  105.81  2.20  0.00  106.64  575.327759   
23591  0.321183  760  30  108.00  469  105.79  2.20  0.00  106.65  576.429749   
23901  0.322657  770  30  107.99  469  105.78  2.21  0.00  106.67  577.022949   
24211  0.324103  780  30  107.98  468  105.76  2.21  0.00  106.68  577.569641   
24521  0.345045  790  30  107.96  468  105.74  2.22  0.00  106.69  577.273682   
24831  0.318661  800  30  107.95  468  105.73  2.22  0.00  106.70  578.336853   
25141  0.422862  810  30  107.94  468  105.71  2.23  0.00  106.71  578.065002   
25451  0.322561  820  30  107.93  467  105.70  2.23  0.00  106.72  579.176941   
25761  0.344857  830  30  107.92  467  105.68  2.23  0.00  106.73  578.892578   
26071  0.301824  840  30  107.91  467  105.67  2.24  0.00  106.75  580.015442   
26381  0.345141  850  30  107.90  467  105.66  2.24  0.00  106.75  579.722351   
26691  0.299431  860  30  107.89  466  105.64  2.24  0.00

0 R: 1.000 P: 0.358 carry pockets strap back ! pocket perfect nice shoulder hold
   1 R: 0.246 P: 0.090 power mouse room pocket cord charger usb nice fine works
     14 R: 0.105 P: 0.105 sleeve ipad inside protection air perfectly snug pro padding bought
     12 R: 0.045 P: 0.045 & ; pro sleeve - retina bought ? room hp
     11 R: 0.007 P: 0.007 ; bought pro perfect love buy ! nice - time
   2 R: 0.131 P: 0.048 months ... broke year years bought design made zipper ago
     24 R: 0.082 P: 0.082 ! love color amazon price pro received back : ordered
   5 R: 0.112 P: 0.034 time price pretty protect bit 'm nice weeks air scratches
     51 R: 0.041 P: 0.041 smell ! sleeve zipper protect air inside odor neoprene pro
     52 R: 0.036 P: 0.036 ! mac love color hard durable pro book material bought
   3 R: 0.153 P: 0.063 bottom top cover feet mac plastic apple easy back piece
     31 R: 0.090 P: 0.090 cover color ! keyboard love pro nice perfectly easily apple
{0: [1, 2, 3, 4], 1: [14, 13], 2: [

TRAIN:                           VALID:              \
           Time   Ep  Ct    LOSS  PPL     NLL    KL   REG    LOSS         PPL   
31     2.768924    0  30  117.72  791  116.80  0.75  0.16  106.72  573.571716   
341    0.617986   10  30  112.66  593  112.25  0.39  0.02  105.61  537.981750   
651    0.300289   20  30  112.06  574  111.59  0.46  0.01  105.47  533.718018   
961    0.739563   30  30  111.63  561  111.03  0.59  0.01  105.29  530.653320   
1271   0.325155   40  30  111.30  552  110.57  0.72  0.01  105.20  530.028931   
1581   0.595456   50  30  111.03  545  110.20  0.83  0.01  105.17  529.619141   
1891   0.548240   60  30  110.81  540  109.89  0.92  0.01  105.22  531.926819   
2201   0.777141   70  30  110.63  535  109.63  1.00  0.01  105.21  531.568298   
2511   0.424271   80  30  110.48  531  109.40  1.07  0.01  105.29  534.278992   
2821   0.353124   90  30  110.34  527  109.20  1.14  0.01  105.31  533.932617   
3131   0.436757  100  30  110.22  524  109.02  1.20  0.01  105.38  536.555054   
3441   0.315981  110  30  110.11  521  108.85  1.25  0.01  105.43  537.874756   
3751   0.324044  120  30  110.01  519  108.70  1.30  0.01  105.47  539.184387   
4061   0.316536  130  30  109.92  517  108.56  1.35  0.01  105.51  540.470947   
4371   0.451363  140  30  109.83  514  108.44  1.39  0.01  105.55  541.683350   
4681   0.332863  150  30  109.76  512  108.32  1.43  0.01  105.58  542.823853   
4991   0.325603  160  30  109.68  511  108.21  1.47  0.01  105.61  543.923767   
5301   0.323954  170  30  109.61  509  108.11  1.50  0.01  105.64  544.965027   
5611   0.310067  180  30  109.55  507  108.01  1.53  0.01  105.67  545.849792   
5921   0.336730  190  30  109.49  506  107.92  1.56  0.01  105.69  546.707031   
6231   0.319263  200  30  109.43  504  107.83  1.59  0.01  105.72  547.496216   
6541   0.374295  210  30  109.38  503  107.75  1.62  0.01  105.74  548.197388   
6851   0.323386  220  30  109.32  502  107.67  1.65  0.01  105.76  548.825623   
7161   0.697526  230  30  109.27  500  107.60  1.67  0.00  105.78  549.404785   
7471   0.776065  240  30  109.22  499  107.53  1.69  0.00  105.81  549.948425   
7781   0.449024  250  30  109.18  498  107.46  1.71  0.00  105.83  550.464722   
8091   0.492245  260  30  109.13  497  107.39  1.73  0.00  105.85  550.964172   
8401   0.452463  270  30  109.09  496  107.33  1.75  0.00  105.88  551.453308   
8711   0.347524  280  30  109.05  495  107.27  1.77  0.00  105.90  551.928955   
9021   0.319563  290  30  109.01  494  107.21  1.79  0.00  105.92  552.394470   
...         ...  ...  ..     ...  ...     ...   ...   ...     ...         ...   
21731  0.342975  700  30  108.07  471  105.90  2.18  0.00  106.57  572.785645   
22041  0.323649  710  30  108.06  471  105.88  2.18  0.00  106.58  573.864685   
22351  0.320522  720  30  108.05  470  105.86  2.19  0.00  106.60  574.487854   
22661  0.362783  730  30  108.04  470  105.84  2.19  0.00  106.62  575.068298   
22971  0.460580  740  30  108.02  470  105.82  2.20  0.00  106.63  575.626343   
23281  0.388145  750  30  108.01  469  105.81  2.20  0.00  106.64  575.327759   
23591  0.321183  760  30  108.00  469  105.79  2.20  0.00  106.65  576.429749   
23901  0.322657  770  30  107.99  469  105.78  2.21  0.00  106.67  577.022949   
24211  0.324103  780  30  107.98  468  105.76  2.21  0.00  106.68  577.569641   
24521  0.345045  790  30  107.96  468  105.74  2.22  0.00  106.69  577.273682   
24831  0.318661  800  30  107.95  468  105.73  2.22  0.00  106.70  578.336853   
25141  0.422862  810  30  107.94  468  105.71  2.23  0.00  106.71  578.065002   
25451  0.322561  820  30  107.93  467  105.70  2.23  0.00  106.72  579.176941   
25761  0.344857  830  30  107.92  467  105.68  2.23  0.00  106.73  578.892578   
26071  0.301824  840  30  107.91  467  105.67  2.24  0.00  106.75  580.015442   
26381  0.345141  850  30  107.90  467  105.66  2.24  0.00  106.75  579.722351   
26691  0.299431  860  30  107.89  466  105.64  2.24  0.00

0 R: 1.000 P: 0.370 carry pockets strap back ! pocket perfect shoulder nice small
   1 R: 0.236 P: 0.085 power mouse cord pocket room charger usb nice works fine
     14 R: 0.093 P: 0.093 sleeve ipad inside protection air perfectly pro padding snug bought
     13 R: 0.058 P: 0.058 & ; pro sleeve - retina bought room ? big
   2 R: 0.129 P: 0.047 ... months year broke bought years design made zipper quality
     24 R: 0.082 P: 0.082 ! love color amazon price : pro received buy back
   3 R: 0.113 P: 0.028 time pretty protect price 'm weeks bit nice scratches air
     31 R: 0.037 P: 0.037 smell ! sleeve zipper protect air inside pro odor days
     32 R: 0.048 P: 0.048 ! mac love color hard durable book pro bought material
   4 R: 0.152 P: 0.067 bottom top cover feet mac plastic apple easy back piece
     41 R: 0.085 P: 0.085 cover color ! keyboard love pro nice perfectly easily apple
